<a href="https://colab.research.google.com/github/gferew1/IntroML/blob/main/homework7part2ofseconditeration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Check for CUDA availability and set the device
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")


CUDA is available. Using GPU.


for question 2 the 2nd of 3 iterations

In [ ]:
# Define the ResNet block
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = self.relu(out)
        return out

# Define the ResNet-10
class ResNet10(nn.Module):
    def __init__(self):
        super(ResNet10, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layers = self._make_layers(ResNetBlock, 64, 10, stride=1)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 10)  # CIFAR-10 has 10 classes

    def _make_layers(self, block, out_channels, num_blocks, stride):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

# Create ResNet-10 model
model = ResNet10().to(device)

# Loss and optimizer with weight decay
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

# Train the model
start_time = time.time()

for epoch in range(300):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/300], Loss: {running_loss / len(train_loader)}')

end_time = time.time()
total_training_time = end_time - start_time

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy}%')

# Report the training time, loss, and accuracy
print(f"Total Training Time: {total_training_time:.2f} seconds")
print(f"Final Training Loss: {running_loss / len(train_loader)}")
print(f"Test Accuracy: {accuracy}%")


100%|██████████| 170498071/170498071 [00:03<00:00, 44318321.20it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/300], Loss: 1.4589160615221008
Epoch [2/300], Loss: 1.0629309109409752
Epoch [3/300], Loss: 0.9194271140696143
Epoch [4/300], Loss: 0.8261904657039496
Epoch [5/300], Loss: 0.7495093525523115
Epoch [6/300], Loss: 0.6790919938050878
Epoch [7/300], Loss: 0.6261995939342567
Epoch [8/300], Loss: 0.5845073655895565
Epoch [9/300], Loss: 0.5480314926113314
Epoch [10/300], Loss: 0.5178459560322335
Epoch [11/300], Loss: 0.49607181503339803
Epoch [12/300], Loss: 0.46853169821717244
Epoch [13/300], Loss: 0.4532507176289473
Epoch [14/300], Loss: 0.4359649487621034
Epoch [15/300], Loss: 0.4218466270457753
Epoch [16/300], Loss: 0.41118787930292244
Epoch [17/300], Loss: 0.3952869180675663
Epoch [18/300], Loss: 0.384347321737148
Epoch [19/300], Loss: 0.3764310587016518
Epoch [20/300], Loss: 0.36497904066844367
Epoch [21/300], Loss: 0.3585716285135435
Epoch [22/300], Loss: 0.3472706419046578
Epoch [23/300],